In [1]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

rank = 0 
sys.path[0] = f'{Path().resolve().parents[rank]}' # mymoduleが上の階層にある場合rankを調整してコメント解除
from mymodule import PipeLine, grid_serch_cv

## グリッドサーチによるハイパーパラメータ調整
グリッドサーチ「なし」と「あり」で効果を確認

グリッドサーチ<span style = "color: orange;">なし</span>の場合

In [2]:
df = pd.read_csv('../data/train.csv')
pipe = PipeLine()
pipe(df)
model = pipe.training(valid='fold_out_split', Model=SVC)

-------------------- 使用された特徴量 --------------------


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,342,1,150,1,3.0
1,55,0,130,394,0,150,0,0.0
2,47,1,110,0,1,120,1,0.0
3,34,1,115,0,1,154,0,0.2
4,54,0,160,201,0,163,0,0.0


-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.750557,0.774892,0.748954,0.761702
test,0.704663,0.773913,0.741667,0.757447


グリッドサーチ<span style = "color: orange;">あり</span>の場合

In [3]:
# sklearnの使用するモデル
model = SVC  # クラスの()いらない

#################### グリッドサーチの探索候補を用意 #################
param_range = [0.001, 0.01, 0.1, 1.0, 10]
param_grid = [{'C': param_range,  # 正則化パラメータの係数を探索
               'kernel': ['linear']},  # Linearカーネル
              {'C': param_range, 
               'gamma': param_range,  # カーネル係数を探索
               'kernel': ['rbf']}]  # rbfカーネル

############################ パイプライン ############################
pipe = PipeLine()
pipe(df)
pack = pipe.fold_out_split()
# グリッドサーチの出番
gs_model = grid_serch_cv(pack, param_grid, model)

-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.817372,0.861751,0.782427,0.820175
test,0.813472,0.850000,0.850000,0.850000


精度が向上してる<br>
※ もともと精度が高いデータセットの場合<span style = "color: orange;">精度が下がる場合がある</span><br>
（パラメータ検証のためデータセットを削っているためと思われ)
>
自作した関数では最適なモデルとパラメータを出力するようになってる


In [4]:
print('ベストなパラメータ: ',gs_model.best_params_)
model = gs_model

ベストなパラメータ:  {'C': 0.1, 'kernel': 'linear'}
